In [35]:
%load_ext autoreload
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
sys.path.append(os.path.join('../src/utils'))
sys.path.append(os.path.join('../src'))
import my_model
import eda_functions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [22]:
DB1_FOLDER = '../data/db1'
PREPARED_DATA_FOLDER = '../data/prepared_data'
MODELS_FOLDER = '../src/models'

In [4]:
catch = pd.read_csv(os.path.join(DB1_FOLDER, 'catch.csv'))
catch['date'] = pd.to_datetime(catch['date'])
display(catch.sort_values(by=['id_ves', 'date']))

,id_ves,date,id_region,id_fish,catch_volume,id_regime,permit,id_own
5644,2,2022-01-10,270,848,0.017,1,2161,1051
6351,2,2022-01-11,270,848,0.020,1,2161,1051
13803,2,2022-01-22,274,849,0.062,1,2327,1252
14234,2,2022-01-22,273,849,1.280,1,2327,1252
14536,2,2022-01-23,274,849,0.054,1,2327,1252
...,...,...,...,...,...,...,...,...
74553,1659,2022-04-09,273,400,106.120,1,2887,1266
76231,1659,2022-04-11,273,204,4.909,1,2887,1266
77132,1659,2022-04-12,273,204,34.017,1,2887,1266
78026,1659,2022-04-13,273,204,34.360,1,2887,1266


In [6]:
product = pd.read_csv(os.path.join(DB1_FOLDER, 'product.csv'))
product['date'] = pd.to_datetime(product['date'])
display(product.sort_values(by=['id_ves', 'date']))

,id_ves,date,id_prod_designate,id_prod_type,prod_volume,prod_board_volume
19946,1,2022-01-07,3,3971,0.0,7.04
19947,1,2022-01-07,3,3917,0.0,1.56
19948,1,2022-01-07,3,3817,0.0,0.28
19949,1,2022-01-07,3,3694,0.0,219.87
19950,1,2022-01-07,3,3524,0.0,70.04
...,...,...,...,...,...,...
378258,1659,2022-04-16,1,4555,0.0,126.41
381180,1659,2022-04-17,1,4555,0.0,126.41
385295,1659,2022-04-18,1,4555,0.0,126.41
390182,1659,2022-04-19,1,4555,0.0,126.41


# Собираем словари для мэппинга айдишников рыб, регионов и проч. с их названиями

In [9]:
prod_designate = pd.read_csv(os.path.join(DB1_FOLDER, 'ref', 'prod_designate.csv'), sep=';')
display(prod_designate)
prod_designate_map = prod_designate.set_index('id_prod_designate')['prod_designate'].to_dict()

,id_prod_designate,prod_designate
0,0,не определен
1,1,"рыбопродукция, выпущенная из собственного сырья (сырца)"
2,3,"рыбопродукция, принятая для транспортировки"
3,4,"рыбопродукция, принятая (купленная) для промпереработки"
4,5,"рыбопродукция из собств сырья, предназн для промпереработки"
5,8,"промпереработанная рыбопродукция,выпущ ранее из собств сырья"
6,2,"рыбопродукция, выпущенная из сырья,полученного от иного лица"


In [10]:
ref_fish = pd.read_csv(os.path.join(DB1_FOLDER, 'ref', 'fish.csv'), sep=';')
display(ref_fish)
ref_fish_map = ref_fish.set_index('id_fish')['fish'].to_dict()

,id_fish,fish
0,0,Не определен
1,1,макруронус
2,2,Рыбы и морепродукты
3,3,Рыбы
4,4,сельди
...,...,...
817,443,лещи морские серебристые
818,172,"скат колючий, лисица морская"
819,174,скаты песчаные
820,494,керчаки


In [11]:
prod_type = pd.read_csv(os.path.join(DB1_FOLDER, 'ref', 'prod_type.csv'), sep=';')
display(prod_type)
id_prod_prod_type_map = prod_type.groupby('id_prod_type')['prod_type'].agg(set).to_dict()
id_fish_prod_type_map = prod_type.groupby('id_fish')['prod_type'].agg(set).to_dict()
id_prod_type_id_fish_map = prod_type.set_index('id_prod_type')['id_fish'].agg(lambda x: set(str(x))).to_dict()
id_fish_id_prod_type_map = prod_type.set_index('id_fish')['id_prod_type'].agg(lambda x: set(str(x))).to_dict()

,id_prod_type,prod_type,id_fish,prod_type_full
0,6060,Рыба охлажденная,2,рыба охлажденная
1,9420,альгинат натрия,29,альгинат натрия
2,6010,Пряный посол и маринады,2,пряный посол и маринады
3,6490,икра прочих рыб,2,икра прочих рыб
4,8302,"Рыба, морепродукты и отходы от разделки рыбы для зверохозяйс",2,рыба морепродукты и отходы от разделки рыбы для зверохозяйств
...,...,...,...,...
10188,9712,краб-стр опил компл конеч в панц вар-мор гр L,294,краб-стригун опилио комплект конечностей в панцире варено-мороженый группа L
10189,9713,краб-стр опил компл конеч в панц вар-мор гр 2L,294,краб-стригун опилио комплект конечностей в панцире варено-мороженый группа 2L
10190,9714,краб-стр опил компл конеч в панц вар-мор гр 3L,294,краб-стригун опилио комплект конечностей в панцире варено-мороженый группа 3L
10191,9715,краб-стр опил компл конеч в панц вар-мор гр 4L,294,краб-стригун опилио комплект конечностей в панцире варено-мороженый группа 4L


In [12]:
display(ref_fish[ref_fish['id_fish'] == 29])
display(prod_type[prod_type['id_fish'] == 29])

,id_fish,fish
29,29,Водоросли и морские травы


,id_prod_type,prod_type,id_fish,prod_type_full
1,9420,альгинат натрия,29,альгинат натрия
13,8493,маннит,29,маннит
14,8410,"агар, агароид",29,"агар,агароид"
322,2580,агар ос очистки,29,агар особой очистки
323,2585,агар мбиол субл,29,агар микробиологический сублимационной сушки
324,2586,агар очищ нат,29,агар очищеннный (натриевая схема)
325,2588,агар пищ,29,агар пищевой
9860,8420,альгинат натрия,29,альгинат натрия


In [13]:
regime = pd.read_csv(os.path.join(DB1_FOLDER, 'ref', 'regime.csv'), sep=';')
display(regime)
regime_map = regime.set_index('id_regime')['regime'].to_dict()

,id_regime,regime
0,0,не определен
1,1,промышленный лов
2,3,научные исследования
3,12,прибрежное рыболовство
4,15,промысел для РФ по международным договорам
5,18,"добыча (вылов) ВБР, ОДУ которых не установлен"
6,22,промысел в ИЭЗ РФ для ин.гос.(по междунар.договорам)
7,41,на инвестиционные цели (пром)
8,61,добыча (вылов) крабов в инвест. целях (пром)
9,97,разрешенный прилов


In [14]:
region = pd.read_csv(os.path.join(DB1_FOLDER, 'ref', 'region.csv'), sep=';')
display(region)
region_map = region.set_index('id_region')['region'].to_dict()

,id_region,region
0,7010,Бирма
1,7009,Бенин
2,7008,Бельгия
3,7007,Барбадос
4,7006,Бангладеш
...,...,...
3325,9005,Район Регулирования Смешанной Российско-Норвежской комиссии по рыболовству
3326,539,Чжанцзяган/Китай (КНР)/
3327,3345,Лиинахамари/СБ/
3328,549,Черное море восточнее меридиана 36°35'00' в.д.


# Считаем посуточную агрегацию для продуктов

In [15]:
product_data = my_model.prepare_product(product)

100%|██████████| 1529/1529 [05:04<00:00,  5.03it/s] 


In [16]:
product_data.head()

,id_ves,date,1_10000,1_10013,1_10019,1_10025,1_10038,1_10039,1_10040,1_10041,1_10056,1_10057,1_10068,1_10077,1_10082,1_10089,1_10090,1_10091,1_101,1_10106,1_10128,1_10130,1_1015,1_1025,1_1027,1_1055,1_1066,1_1076,1_1092,1_1101,1_1102,1_1113,1_1124,1_1126,1_1135,1_1137,1_1169,1_1173,1_1175,1_1176,1_1184,1_1185,1_1186,1_1195,1_1209,1_1227,1_1228,1_1241,1_1249,1_1254,1_1273,1_1284,1_1285,1_1286,1_1288,1_1292,1_1300,1_1303,1_1311,1_1320,1_1331,1_1337,1_1340,1_1346,1_1350,1_1354,1_1355,1_1356,1_1357,1_1389,1_1391,1_1405,1_1434,1_1450,1_1471,1_1472,1_1473,1_1474,1_1475,1_1551,1_1593,1_1617,1_1619,1_1621,1_1636,1_1637,1_1648,1_1699,1_1702,1_1707,1_1729,1_1730,1_1731,1_1744,1_1755,1_1768,1_188,1_189,1_1892,1_1920,1_1921,1_1923,1_1928,1_1943,1_1988,1_2008,1_2012,1_2014,1_2017,1_2044,1_2050,1_2067,1_2076,1_2081,1_2104,1_2143,1_2145,1_2149,1_2150,1_2154,1_2171,1_2177,1_2194,1_2207,1_2209,1_2211,1_2215,1_2217,1_2269,1_2272,1_2281,1_2283,1_2289,1_2292,1_2298,1_2303,1_2304,1_2305,1_2337,1_2431,1_2456,1_2471,1_2472,1_2509,1_2526,1_2555,1_2598,1_2599,1_2606,1_2607,1_2609,1_2620,1_2634,1_2640,1_2670,1_2671,1_2674,1_2675,1_2676,1_2685,1_2691,1_2715,1_2716,1_2802,1_2805,1_2912,1_2993,1_2995,1_3007,1_3008,1_3021,1_3038,1_3039,1_3074,1_3081,1_3082,1_3084,1_3096,1_3098,1_3101,1_3136,1_3140,1_3142,1_3164,1_3205,1_3211,1_3215,1_3216,1_3217,1_3224,1_3230,1_3231,1_3249,1_3269,1_3286,1_3287,1_3288,1_3289,1_3291,1_3350,1_3368,1_3401,1_3432,1_3433,1_3438,1_3440,1_3441,1_3444,1_3480,1_3494,1_3500,1_3501,1_3505,1_3506,1_3507,1_3508,1_3512,1_3524,1_3530,1_3531,1_3534,1_3539,1_3602,1_3604,1_3606,1_3612,1_3615,1_3627,1_3630,1_3631,1_3633,1_3644,1_3654,1_3681,1_3690,1_3694,1_3725,1_3726,1_3727,1_3730,1_3731,1_3732,1_3735,1_3741,1_3757,1_3761,1_3769,1_3772,1_3773,1_3778,1_3794,1_3803,1_3806,1_3807,1_3808,1_3811,1_3812,1_3815,1_3817,1_3847,1_3848,1_3880,1_3882,1_3893,1_3897,1_3901,1_3902,1_3903,1_3904,1_3905,1_3912,1_3913,1_3918,1_3934,1_3939,1_3942,1_3947,1_3957,1_3959,1_3964,1_3966,1_3968,1_3970,1_3972,1_3974,1_3976,1_3981,1_3995,1_3996,1_3999,1_4000,1_4005,1_4007,1_4008,1_4009,1_4010,1_4012,1_4014,1_4029,1_4030,1_4033,1_4039,1_4040,1_4044,1_4045,1_4046,1_4047,1_4048,1_4049,1_4050,1_4051,1_4052,1_4053,1_4054,1_406068,1_4061,1_4065,1_4070,1_4079,1_4080,1_4084,1_4090,1_4095,1_4096,1_4103,1_4108,1_4121,1_4123,1_4124,1_4125,1_4127,1_4129,1_4138,1_4141,1_4147,1_4152,1_4154,1_4167,1_4168,1_4169,1_4174,1_4175,1_4181,1_4182,1_4184,1_4186,1_4195,1_4208,1_4209,1_4210,1_4211,1_4213,1_4217,1_4232,1_4236,1_4253,1_4254,1_4255,1_4269,1_4270,1_4271,1_4281,1_4290,1_4291,1_4302,1_4303,1_4304,1_4305,1_4307,1_4315,1_4319,1_4326,1_4344,1_4345,1_4347,1_4348,1_4349,1_4350,1_4352,1_4374,1_4375,1_4381,1_4401,1_4408,1_4409,1_4410,1_4411,1_4412,1_4413,1_4420,1_4422,1_4446,1_4450,1_4451,1_4452,1_4484,1_4486,1_4502,1_4504,1_4506,1_4510,1_4512,1_4515,1_4516,1_4517,1_4518,1_4522,1_4526,1_4532,1_4537,1_4548,1_4553,1_4555,1_4558,1_4565,1_4567,1_4568,1_4579,1_4580,1_4581,1_4582,1_4583,1_4584,1_4590,1_4591,1_4592,1_4594,1_4595,1_4596,1_4611,1_4614,1_4615,1_4625,1_4626,1_4627,1_4628,1_4629,1_4636,1_4637,1_4638,1_4639,1_4646,1_4648,1_4652,1_4674,1_4707,1_4729,1_4730,1_4731,1_4732,1_4733,1_4734,1_4740,1_4741,1_4742,1_4743,1_4744,1_4745,1_4746,1_4747,1_4748,1_4749,1_4761,1_4762,1_4764,1_4769,1_4785,1_4786,1_4787,1_4791,1_4795,1_4820,1_4825,1_4828,1_4829,1_4831,1_4833,1_4835,1_4838,1_4839,1_4841,1_4843,1_4845,1_4846,1_4850,1_4853,1_4869,1_4875,1_4883,1_4885,1_4893,1_4895,1_4913,1_4914,1_4916,1_4918,1_4919,1_4926,1_4940,1_4942,1_4944,1_4946,1_4948,1_4962,1_4966,1_4968,1_4976,1_4977,1_4978,1_4979,1_4980,1_4981,1_4982,1_4983,1_4985,1_4988,1_4989,1_4992,1_4993,1_5009,1_5010,1_5011,1_5018,1_5054,1_5055,1_5066,1_5072,1_5073,1_5081,1_5087,1_5090,1_5091,1_5092,1_5093,1_5100,1_5230,1_5241,1_5258,1_5262,1_5263,1_5264,1_5270,1_5272,1_5274,1_5321,1_5322,1_5326,1_5331,1_5336,1_5346,1_5365,1_5374,1_5375,1_5376,1_5416,1_5418,1_5427,1_5433,1_5444,1_5455,1_5458,1_5465,1_5466,1_5467,1_5468,1_5471,1_5485,1_5508,1_5517,

# Считаем посуточную агрегацию для рыбы

In [17]:
catch_data = my_model.prepare_catch(catch)
catch_data.head()

,id_ves,date,6_fish,88_fish,90_fish,91_fish,112_fish,113_fish,114_fish,124_fish,125_fish,128_fish,144_fish,147_fish,150_fish,166_fish,167_fish,172_fish,173_fish,174_fish,188_fish,203_fish,204_fish,210_fish,222_fish,224_fish,228_fish,229_fish,233_fish,249_fish,251_fish,252_fish,253_fish,254_fish,286_fish,291_fish,292_fish,294_fish,304_fish,340_fish,352_fish,377_fish,399_fish,400_fish,401_fish,403_fish,404_fish,408_fish,409_fish,410_fish,411_fish,414_fish,431_fish,434_fish,439_fish,451_fish,453_fish,457_fish,469_fish,473_fish,487_fish,508_fish,510_fish,516_fish,551_fish,556_fish,558_fish,571_fish,572_fish,591_fish,610_fish,614_fish,616_fish,617_fish,631_fish,639_fish,645_fish,650_fish,672_fish,677_fish,683_fish,692_fish,696_fish,713_fish,714_fish,717_fish,720_fish,721_fish,729_fish,731_fish,733_fish,734_fish,738_fish,755_fish,760_fish,779_fish,821_fish,833_fish,839_fish,844_fish,847_fish,848_fish,849_fish,853_fish,854_fish,859_fish,860_fish,861_fish,878_fish,880_fish,882_fish,886_fish,887_fish,888_fish,889_fish,927_fish,946_fish,955_fish,978_fish,984_fish,0_regime,1_regime,3_regime,12_regime,15_regime,18_regime,22_regime,41_regime,61_regime,97_regime,108_region,110_region,116_region,129_region,130_region,131_region,162_region,199_region,206_region,207_region,261_region,264_region,265_region,266_region,267_region,268_region,269_region,270_region,271_region,272_region,273_region,274_region,275_region,277_region,278_region,285_region,298_region,304_region,310_region,387_region,399_region,436_region,437_region,473_region,501_region,502_region,507_region,513_region,521_region,526_region,531_region,547_region,548_region,549_region,550_region,2775_region,2776_region,5432_region,5434_region,5435_region
0,2,2022-01-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,2022-01-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,2022-01-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,1.342,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2,2022-01-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,2.078,0.0,0.0,0.0,0.0,0.0,0.

# Мерджим и сохраняем в pickle

In [18]:
merged = my_model.merge_prepared_catch_product(catch_data, product_data)

In [19]:
merged.to_pickle(os.path.join(PREPARED_DATA_FOLDER, 'catch_product_merged.pickle'))

In [27]:
needed_columns = {}
needed_columns['catch'] = list(catch_data.columns)
needed_columns['product'] = list(product_data.columns)

filepath = os.path.join(MODELS_FOLDER, 'needed_columns.json')
with open(filepath, 'w') as rec_file:
    json.dump(needed_columns, rec_file)

In [30]:
catch_test = catch[:100].copy()
product_test = product[product['id_ves'].isin(catch_test['id_ves']) & product['date'].isin(catch_test['date'])].copy()

In [58]:
%autoreload 2
test_data = \
    my_model.prepare_data_for_model(catch_test,
                                    product_test,
                                    key_cols=None,
                                    verbose=True,
                                    needed_cols_path='../src/models/needed_columns.json')

В product отсутствовали колонки ['6_fish', '90_fish', '91_fish', '124_fish', '125_fish', '128_fish', '150_fish', '166_fish', '167_fish', '172_fish', '173_fish', '174_fish', '188_fish', '203_fish', '210_fish', '222_fish', '228_fish', '229_fish', '233_fish', '249_fish', '251_fish', '252_fish', '253_fish', '254_fish', '286_fish', '291_fish', '304_fish', '340_fish', '352_fish', '377_fish', '399_fish', '401_fish', '403_fish', '404_fish', '410_fish', '411_fish', '414_fish', '431_fish', '434_fish', '439_fish', '453_fish', '457_fish', '469_fish', '473_fish', '487_fish', '508_fish', '510_fish', '516_fish', '551_fish', '556_fish', '558_fish', '571_fish', '572_fish', '591_fish', '614_fish', '631_fish', '639_fish', '645_fish', '650_fish', '672_fish', '677_fish', '692_fish', '713_fish', '717_fish', '720_fish', '731_fish', '733_fish', '734_fish', '738_fish', '755_fish', '760_fish', '779_fish', '821_fish', '833_fish', '839_fish', '844_fish', '848_fish', '849_fish', '853_fish', '854_fish', '859_fish',

100%|██████████| 170/170 [00:00<00:00, 2548.23it/s]

В product отсутствовали колонки ['1_10000', '1_10013', '1_10019', '1_10038', '1_10040', '1_10041', '1_10056', '1_10057', '1_10068', '1_10077', '1_10082', '1_10089', '1_10090', '1_10091', '1_101', '1_10130', '1_1015', '1_1025', '1_1027', '1_1055', '1_1066', '1_1076', '1_1102', '1_1113', '1_1124', '1_1126', '1_1137', '1_1169', '1_1176', '1_1184', '1_1185', '1_1195', '1_1209', '1_1227', '1_1228', '1_1241', '1_1249', '1_1254', '1_1284', '1_1285', '1_1286', '1_1288', '1_1292', '1_1300', '1_1303', '1_1311', '1_1320', '1_1331', '1_1337', '1_1340', '1_1346', '1_1350', '1_1354', '1_1356', '1_1357', '1_1389', '1_1391', '1_1434', '1_1471', '1_1475', '1_1617', '1_1619', '1_1621', '1_1636', '1_1637', '1_1648', '1_1699', '1_1702', '1_1707', '1_1729', '1_1730', '1_1731', '1_1744', '1_1755', '1_188', '1_189', '1_1892', '1_1920', '1_1921', '1_1923', '1_1928', '1_1943', '1_1988', '1_2008', '1_2012', '1_2014', '1_2017', '1_2044', '1_2050', '1_2067', '1_2076', '1_2081', '1_2104', '1_2145', '1_2149', '1_21